In [ ]:
---
title: 1D string vibration - Part II
description: Method of characteristics
author: Daning H.
show-code: False
show-prompt: False
params:
    c:
        input: numeric
        label: Wave speed
        value: 1.0
        step: 1.0
    p1:
        input: checkbox
        label: Right-running solution
        input: checkbox
        value: True
    p2:
        input: checkbox
        label: Left-running solution
        input: checkbox
        value: True
    p3:
        input: checkbox
        label: D'Alembert solution
        input: checkbox
        value: True
---

We solve a 1D string vibration problem with fixed ends on the domain $0\leq x\leq \pi,\, t\geq 0$
$$
\begin{aligned}
&\, u_{tt} = c^2u_{xx} \\
&\, u(0,t) = 0,\quad u(\pi,t) = 0, \\
&\, u(x,0)=f(x),\quad u_t(x,0)=0
\end{aligned}
$$
where $c$ is the **wave speed** and see figure below for the definition of initial displacement $f(x)$.  The initial velocity is assumed to be zero.

Here we solve the problem by the **method of characteristics**, and by D'Alembert solution,
$$
u(x,t) = \frac{1}{2} \left( \hat{f}(x+ct) + \hat{f}(x-ct) \right)
$$
where $\hat{f}(x)$ is the **odd extension** of the initial displacement $f(x)$.  The physical picture is that, at the start of time, the initial displacement splits evenly into two and run left and right, respectively, both at speed $c$.

When interacting with the different solutions, think about:
+ What is the effect of $c$?  Why do we call it wave speed?

In [ ]:
c = 1.0
p1 = True
p2 = True
p3 = True

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
from IPython.display import HTML

def oddExt(f, P):
    def _fOdd(x):
        _x0 = np.array(x)
        _y  = np.zeros_like(_x0)
        _x  = np.mod(_x0, 2*P)
        _m  = _x <= P
        _y[_m] = f(_x[_m])
        _y[~_m] = -f(2*P-_x[~_m])
        return _y
    return _fOdd

bnd = np.pi
Nt  = 51
xlm = 0.3

fd = lambda x: 0.1*x*(bnd-x)
od = oddExt(fd, bnd)

x  = np.linspace(-2*bnd, 3*bnd, 101)
x0 = np.linspace(0.0, bnd, 101)
t  = np.linspace(0.0, 20*bnd/c, 10*Nt)
X, T = np.meshgrid(x, t)
X0, T0 = np.meshgrid(x0, t)
f1 = 0.5*od(X-c*T)
f2 = 0.5*od(X+c*T)
fs = 0.5*od(X0-c*T0) + 0.5*od(X0+c*T0)

fig = plt.figure()
ax = plt.gca()
ls = [None, None, None, None]
plt.plot(x, 2*f1[0], 'g-.', linewidth=0.5, label=r'IC: $u(x,0)=f(x)$')
if p1:
    ls[0], = plt.plot([], [], 'r--', label=r'$\frac{1}{2}f(x-ct)$')
if p2:
    ls[1], = plt.plot([], [], 'b--', label=r'$\frac{1}{2}f(x+ct)$')
if p3:
    ls[2], = plt.plot([], [], 'k-',  label=r'$\frac{1}{2}f(x-ct)+\frac{1}{2}f(x+ct)$',
                      linewidth=0.5)
    ls[3], = plt.plot([], [], 'k-',  label='String vibration')
l0, = plt.plot([], [], 'k-')
plt.plot([0.0, 0.0], [-xlm, xlm], 'k--', linewidth=0.5)
plt.plot([bnd, bnd], [-xlm, xlm], 'k--', linewidth=0.5)
plt.plot([0.0, bnd],  [0.0, 0.0], 'k--', linewidth=0.5)
plt.xlabel('x')
plt.xlim([-2*bnd, 3*bnd])
plt.ylabel('u')
plt.ylim([-xlm, xlm])
plt.legend(loc=1)

def animate(i):
    if ls[0] is not None:
        ls[0].set_data(x, f1[i])
    if ls[1] is not None:
        ls[1].set_data(x, f2[i])
    if ls[2] is not None:
        ls[2].set_data(x, f1[i]+f2[i])
    if ls[3] is not None:
        ls[3].set_data(x0, fs[i])
    ax.set_title("t={0}T".format(i/(Nt-1.0)))
    return l0,  # A hack here: return an irrelevant line to update the rest of lines

ani = animation.FuncAnimation(fig, animate, Nt, interval=50, blit=True)
plt.close(ani._fig)
HTML(ani.to_html5_video())